# DeepSpeed 학습 가이드 (간소화)

## DeepSpeed란 무엇인가?

### 1. DeepSpeed의 개념
DeepSpeed는 Microsoft에서 개발한 딥러닝 최적화 라이브러리입니다. 대규모 모델을 효율적으로 학습시키기 위해 만들어졌으며, 특히 LLM(Large Language Model) 학습에 매우 유용합니다.

### 2. 왜 DeepSpeed가 필요한가?

#### 문제 상황
- 현대의 LLM은 수십억 개의 파라미터를 가집니다
- 이런 큰 모델을 학습하려면 엄청난 GPU 메모리가 필요합니다
- 일반적인 GPU 하나로는 메모리가 부족합니다
- 학습 속도가 매우 느립니다

#### DeepSpeed의 해결책
DeepSpeed는 다음과 같은 기술들을 통해 이 문제들을 해결합니다:

1. **메모리 최적화**: 모델의 다른 부분들을 여러 GPU에 나누어 저장
2. **계산 최적화**: 병렬 처리를 통해 학습 속도 향상
3. **통신 최적화**: GPU 간 데이터 전송을 효율화

### 3. DeepSpeed의 주요 기능들

#### ZeRO (Zero Redundancy Optimizer)
- **ZeRO Stage 1**: 옵티마이저 상태를 여러 GPU에 분산
- **ZeRO Stage 2**: 그래디언트도 여러 GPU에 분산
- **ZeRO Stage 3**: 모델 파라미터까지 여러 GPU에 분산

#### 3D Parallelism
- **데이터 병렬**: 같은 모델을 여러 GPU에서 다른 데이터로 학습
- **모델 병렬**: 모델 자체를 여러 GPU에 나누어 배치
- **파이프라인 병렬**: 모델의 레이어들을 순차적으로 여러 GPU에 배치

### 4. DeepSpeed vs 일반 학습 비교

| 구분 | 일반 학습 | DeepSpeed |
|------|-----------|-----------| 
| 메모리 사용량 | 높음 | 낮음 (최대 80% 절약) |
| 학습 속도 | 느림 | 빠름 (여러 GPU 활용) |
| 모델 크기 제한 | GPU 메모리에 의존 | 훨씬 큰 모델 가능 |
| 설정 복잡도 | 간단 | 초기 설정 필요 |


In [1]:
# 1단계: 필요한 라이브러리 설치
# MPI 라이브러리 설치 (DeepSpeed 오류 해결용)
!apt-get update -qq
!apt-get install -y openmpi-bin openmpi-common libopenmpi-dev

# Python 패키지 설치
!pip install deepspeed transformers datasets accelerate mpi4py

# MPI 설치 확인
!which mpirun
!mpirun --version

print("라이브러리 설치가 완료되었습니다!")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopenmpi-dev is already the newest version (4.1.2-2ubuntu1).
openmpi-bin is already the newest version (4.1.2-2ubuntu1).
openmpi-common is already the newest version (4.1.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 92 not upgraded.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
/usr/bin/mpirun
mpirun (Open MPI) 4.1.2

Report bugs to http://www.open-mpi.org/community/help/
라이브러리 설치가 완료되었습니다!


In [2]:
# 2단계: 라이브러리 임포트
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
import json
import os

print("모든 라이브러리가 성공적으로 임포트되었습니다!")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")


모든 라이브러리가 성공적으로 임포트되었습니다!
PyTorch 버전: 2.8.0.dev20250319+cu128
CUDA 사용 가능: True


In [3]:
# 3단계: 한국어 모델과 토크나이저 로드 (device_map 없이)
print("한국어 모델을 로드합니다...")

# 한국어 특화 모델 선택
model_name = "EleutherAI/polyglot-ko-1.3b"  # 1.3B 파라미터

try:
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # 모델 로드 (device_map 사용 안함 - DeepSpeed 호환)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        dtype=torch.float16,
        trust_remote_code=True
    )
    
    # GPU로 이동
    if torch.cuda.is_available():
        model = model.cuda()
        print(f"모델이 GPU로 이동: {next(model.parameters()).device}")
    
    print("한국어 모델 로드 완료!")
    print(f"모델 파라미터 수: {model.num_parameters():,}")
    
except Exception as e:
    print(f"한국어 모델 로드 실패: {e}")
    print("GPT-2 모델을 대안으로 사용합니다...")
    
    model_name = "gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        dtype=torch.float16
    )
    
    if torch.cuda.is_available():
        model = model.cuda()
    
    print("GPT-2 모델로 설정 완료!")


한국어 모델을 로드합니다...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

모델이 GPU로 이동: cuda:0
한국어 모델 로드 완료!
모델 파라미터 수: 1,331,810,304


In [4]:
# 4단계: 한국어 학습 데이터 준비
korean_texts = [
    "인공지능은 우리 일상을 변화시키는 혁신적인 기술입니다. 스마트폰의 음성인식부터 자율주행차까지 다양한 분야에서 활용되고 있습니다.",
    "딥러닝은 인간의 뇌 구조를 모방한 신경망 기술로, 복잡한 패턴 인식과 학습이 가능합니다. 이미지 인식, 자연어 처리 등에서 놀라운 성과를 보여주고 있습니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성할 수 있게 하는 기술입니다. 번역, 요약, 질문답변 등 다양한 언어 작업을 자동화할 수 있습니다.",
    "트랜스포머 아키텍처는 현대 자연어 처리의 핵심 기술입니다. 어텐션 메커니즘을 통해 문맥 정보를 효과적으로 처리할 수 있습니다.",
    "파인튜닝은 사전 훈련된 모델을 특정 작업에 맞게 조정하는 과정입니다. 적은 데이터로도 높은 성능을 달성할 수 있는 효율적인 방법입니다.",
    "한국의 사계절은 각각 독특한 아름다움을 가지고 있습니다. 봄의 벚꽃, 여름의 녹음, 가을의 단풍, 겨울의 설경이 모두 매력적입니다.",
    "한국 음식은 발효 문화를 바탕으로 한 건강한 식문화를 자랑합니다. 김치, 된장, 고추장 등은 세계적으로 인정받는 발효식품입니다.",
    "K-팝과 K-드라마는 한류의 중심으로 전 세계에 한국 문화를 알리고 있습니다. 음악과 영상 콘텐츠의 품질이 매우 높습니다."
]

print(f"총 한국어 학습 텍스트 수: {len(korean_texts)}")

# 텍스트 토큰화
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

# 데이터셋 생성
train_dataset = Dataset.from_dict({"text": korean_texts})
train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

print(f"학습 데이터셋 크기: {len(train_dataset)}")


총 한국어 학습 텍스트 수: 8


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

학습 데이터셋 크기: 8


In [5]:
# 5단계: 간단한 학습 실행 (DeepSpeed 대신 일반 Trainer 사용)
print("간단한 설정으로 학습을 시작합니다...")
print("=" * 50)

# 간단한 TrainingArguments
training_args = TrainingArguments(
    output_dir="./simple_output",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    save_steps=10,
    logging_steps=1,
    learning_rate=5e-5,
    warmup_steps=1,
    weight_decay=0.01,
    fp16=True,
    dataloader_num_workers=0,
    remove_unused_columns=False,
    report_to=None,
)

# 데이터 콜레이터
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    processing_class=tokenizer,
)

# 학습 전 메모리 확인
if torch.cuda.is_available():
    print(f"학습 전 GPU 메모리: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

# 학습 실행
try:
    print("학습을 시작합니다...")
    trainer.train()
    
    print("=" * 50)
    print("✅ 학습이 성공적으로 완료되었습니다!")
    
    if torch.cuda.is_available():
        print(f"학습 후 GPU 메모리: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
        
except Exception as e:
    print(f"❌ 학습 실패: {e}")
    import traceback
    traceback.print_exc()


간단한 설정으로 학습을 시작합니다...
[2025-09-03 10:58:13,696] [INFO] [real_accelerator.py:260:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


[2025-09-03 10:58:15,335] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 2}.


학습 전 GPU 메모리: 2541.23 MB
학습을 시작합니다...


Step,Training Loss
1,1.074900


✅ 학습이 성공적으로 완료되었습니다!
학습 후 GPU 메모리: 7640.07 MB


In [6]:
# 6단계: 학습된 모델 테스트
print("학습된 모델을 테스트합니다...")
print("=" * 50)

# 한국어 테스트 프롬프트들
test_prompts = [
    "인공지능은",
    "한국의 전통문화는",
    "딥러닝 기술의 미래는"
]

# 각 프롬프트에 대해 텍스트 생성
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n테스트 {i}: '{prompt}'")
    
    try:
        # 입력 토큰화
        inputs = tokenizer.encode(prompt, return_tensors="pt")
        
        if torch.cuda.is_available():
            inputs = inputs.cuda()
        
        # 텍스트 생성
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_length=70,
                num_return_sequences=1,
                temperature=0.7,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                top_p=0.9,
                no_repeat_ngram_size=2
            )
        
        # 생성된 텍스트 디코딩
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"생성 결과: {generated_text}")
        
    except Exception as e:
        print(f"생성 실패: {e}")

print("\n" + "=" * 50)
print("🎉 한국어 모델 파인튜닝 완료!")
print("\n배운 내용:")
print("1. DeepSpeed 없이도 효과적인 파인튜닝 가능")
print("2. device_map 문제 해결 방법")
print("3. 한국어 모델 사용법")
print("4. 실용적인 학습 설정")


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


학습된 모델을 테스트합니다...

테스트 1: '인공지능은'
생성 결과: 인공지능은 미래의 인류를 어떻게 바꿀 것인가? - 1부 : 4차 산업혁명의 시작, 인공지능의 미래 - 2부: 인공지능, 인간지능의 확장 - 3부-4부 - 5부 인공지능이 인간의 일자리를 빼앗는가? 인간이 인공지능에게 일자리 빼앗길까? 인공지능 시대의 새로운 일자리, 4가지

테스트 2: '한국의 전통문화는'
생성 결과: 한국의 전통문화는 유교의 정신문화이다. 유교는 모든 사람이 평등하다고 가르치고 있다”며 “우리는 이러한 유교를 계승하고 발전시켜 나가야 한다”고 강조했다(『한국유교의 이해』, p.14). 그는 또 “유교도 유교지만 유교에서 말하는

테스트 3: '딥러닝 기술의 미래는'
생성 결과: 딥러닝 기술의 미래는?​- AI는 인간의 학습능력과 추론능력, 지각능력을 기계적 알고리즘으로 구현한 기술.- 인간이 할 수 없는 일을 AI가 대신함으로써 인간은 더 많은 시간을 자유롭게 사용할 수 있고, 그로 인한 경제적, 사회적 발전

🎉 한국어 모델 파인튜닝 완료!

배운 내용:
1. DeepSpeed 없이도 효과적인 파인튜닝 가능
2. device_map 문제 해결 방법
3. 한국어 모델 사용법
4. 실용적인 학습 설정


In [7]:
# 7단계: 모델 저장
print("학습된 모델을 저장합니다...")

try:
    # 모델과 토크나이저 저장
    model.save_pretrained("./finetuned_korean_model")
    tokenizer.save_pretrained("./finetuned_korean_model")
    
    print("✅ 모델이 성공적으로 저장되었습니다!")
    print("저장 위치: ./finetuned_korean_model")
    
except Exception as e:
    print(f"❌ 모델 저장 실패: {e}")
    
print("\n파인튜닝 실습이 완료되었습니다! 🎉")


학습된 모델을 저장합니다...
✅ 모델이 성공적으로 저장되었습니다!
저장 위치: ./finetuned_korean_model

파인튜닝 실습이 완료되었습니다! 🎉
